1. import whole dataset to get overview - find out that sentences are not macthed here

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [7]:
N = 100
with open('drive/My Drive/Paraphrasing API/datasets/Quora/quora_duplicate_questions.tsv', "r") as file:  # the a opens it in append mode
    for i in range(N):
        line = next(file).strip()
        print(line)

id	qid1	qid2	question1	question2	is_duplicate
0	1	2	What is the step by step guide to invest in share market in india?	What is the step by step guide to invest in share market?	0
1	3	4	What is the story of Kohinoor (Koh-i-Noor) Diamond?	What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?	0
2	5	6	How can I increase the speed of my internet connection while using a VPN?	How can Internet speed be increased by hacking through DNS?	0
3	7	8	Why am I mentally very lonely? How can I solve it?	Find the remainder when [math]23^{24}[/math] is divided by 24,23?	0
4	9	10	Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?	Which fish would survive in salt water?	0
5	11	12	Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?	I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?	1
6	13	14	Should I buy tiago?	What keeps childern active and far from phone and video games?	

In [8]:
data = open('drive/My Drive/Paraphrasing API/datasets/Quora/quora_duplicate_questions.tsv', "r")

In [9]:
df = pd.DataFrame()
for chunk in pd.read_table(data, header=6, names=['id',	'qid1',	'qid2',	'question1',	'question2',	'is_duplicate'], chunksize=1000):
    df = pd.concat([df, chunk], ignore_index=True)

In [25]:
df['question1'] = df['question1'].astype(str)

In [26]:
df['question2'] = df['question2'].astype(str)

In [27]:
df

,id,qid1,qid2,question1,question2,is_duplicate
0,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
1,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
2,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
3,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0
4,10,21,22,Method to find separation of slits using fresn...,What are some of the things technicians can te...,0
...,...,...,...,...,...,...
404279,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404280,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404281,404287,537928,537929,What is one coin?,What's this coin?,0
404282,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [28]:
len(df)

404284

In [29]:
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [30]:
df_train, df_test, df_eval = train_validate_test_split(df)

In [31]:
df_train

,id,qid1,qid2,question1,question2,is_duplicate
162973,162979,165226,253623,What is the relationship between philosophy an...,What is the relationship between education and...,1
340854,340860,468636,468637,How pure or effective are the medicines and pr...,Is it true that all Patanjali products are pur...,0
80362,80368,20852,136646,"After 2 years of physical relationships, my bo...","Why would he tell me he loves me, that I make ...",0
333198,333204,419413,72384,Can non US citizens receive social security be...,Can a non US citizen get social security?,1
403545,403551,12023,7155,What are your opinions on Brexit?,What is your view/opinion about Brexit?,1
...,...,...,...,...,...,...
126747,126753,17126,62975,How did Donald Trump win your vote?,Why did Donald Trump win the 2016 American ele...,1
220827,220833,18866,301010,What age is considered a senior citizen? What ...,What age is considered senior citizen?,1
46582,46588,23171,39977,Why did Trump win the Presidency?,How is Donald Trump winning?,1
337828,337834,465338,465339,If a man was abusive to his girlfriend would m...,"As a woman, if you were to receive the most be...",0


In [32]:
df_train.to_csv('drive/My Drive/Paraphrasing API/datasets/Quora/quora_train.csv')

In [33]:
df_test.to_csv('drive/My Drive/Paraphrasing API/datasets/Quora/quora_test.csv')

In [34]:
df_eval.to_csv('drive/My Drive/Paraphrasing API/datasets/Quora/quora_eval.csv')

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.3 MB/s 


In [5]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 7.3 MB/s 
     |████████████████████████████████| 3.3 MB 33.9 MB/s 
     |████████████████████████████████| 596 kB 15.1 MB/s 
     |████████████████████████████████| 895 kB 58.4 MB/s 
     |████████████████████████████████| 59 kB 7.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import sentencepiece
from transformers import T5Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

In [7]:
print(tokenizer)

PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>',

In [45]:
tokens_input = tokenizer(df_eval.question2.map(lambda x: x).max())

In [46]:
print(len(tokens_input['input_ids']))

11


In [9]:
from transformers import T5ForConditionalGeneration

In [10]:
model_final = T5ForConditionalGeneration.from_pretrained('drive/My Drive/Paraphrasing API/models/Flora_T5 Quora fine-tuned/model')

404 Client Error: Not Found for url: https://huggingface.co/drive/My%20Drive/Paraphrasing%20API/models/Flora_T5%20Quora%20fine-tuned/model/resolve/main/config.json


OSError: ignored